# Target Market Analysis

## Investment Analysis

### House pricing increase compare

In [7]:
import pandas as pd

# Assuming your CSV file is named 'data.csv' and contains a column named 'date' with dates in 'YYYY-MM-DD' format
df = pd.read_csv('/Users/bichengwang/Documents/codes/python-notebook/src/finance/data/SPX GOOGLEFINANCE.csv', date_parser=pd.to_datetime)

# Now df contains your data with the 'date' column parsed as datetime objects
print(df.head())

                 Date   Close
0   1/5/1990 16:00:00  352.20
1  1/12/1990 16:00:00  339.93
2  1/19/1990 16:00:00  339.15
3  1/26/1990 16:00:00  325.80
4   2/2/1990 16:00:00  330.92


/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/735624823.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('/Users/bichengwang/Documents/codes/python-notebook/src/finance/data/SPX GOOGLEFINANCE.csv', date_parser=pd.to_datetime)


In [37]:
import numpy as np

In [52]:
df['Diff'] = df['Close'] - df['Close'].shift(1)
# 300, 310, 305 -> 
# NaN, 300, 310 -> 
# agent() -> 

def buy_and_sell(df, annual_risk_free_rate=0.06, span=4):
    # span lookback weeks
    # risk free rate
    df['Moving_Average'] = df['Close'].rolling(window=span).mean()
    df['EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
    df['Mark'] = df['Close'] > df['EMA']
    marks = df['Mark'].values
    curr = marks[0]
    prices = df['Close'].values
    spx_returns = (df['Close'] / prices[0]).values
    base = 1.
    share = 0.
    security_returns = []
    strategy = []
    ### buy in strategy
    for mark, price in zip(marks, prices):
        if mark != curr:
            if not mark:
                share += base / price
                base = 0.
                strategy.append('Buy')
            else:
                base += share * price
                share = 0.
                strategy.append('Sell')
        else:
            strategy.append('Hold')
        curr = mark
        security_returns.append(base + share * price)
    ratio = (base + share * prices[-1]) / (prices[-1] / prices[0])

    # Calculate the mean returns
    security_mean_return = np.mean(security_returns)
    spx_mean_return = np.mean(spx_returns)

    # Calculate the covariance and variance of returns
    covariance = np.cov(security_returns, spx_returns)[0, 1]
    variance_spx = np.var(spx_returns)

    # Calculate beta
    beta = covariance / variance_spx

    # Calculate alpha
    alpha = security_mean_return - (beta * spx_mean_return)

    # Calculate the excess return of the security over the risk-free rate
    # Sample weekly returns of the security and risk-free rate
    annual_security_returns =  np.array([y/x for x, y in zip(security_returns[:-52], security_returns[52:])])

    # Calculate the excess return of the security over the risk-free rate
    excess_returns = annual_security_returns - annual_risk_free_rate

    # Calculate the annualized average excess return
    annualized_excess_return = np.mean(excess_returns)  # Approximate number of weeks in a year

    # Calculate the annualized standard deviation of returns
    annualized_std_dev = np.std(annual_security_returns) # Approximate number of weeks in a year

    # Calculate the annual Sharpe Ratio
    annual_sharpe_ratio = annualized_excess_return / annualized_std_dev

    def calculate_maximum_withdrawal_rate(weekly_returns, target_years, tolerance=1e-6):
        withdrawal_rate = 0.01  # Starting withdrawal rate
        while True:
            # Calculate portfolio balance after target_years with current withdrawal rate
            balance = calculate_portfolio_balance(weekly_returns, withdrawal_rate)
            if balance > 0:  # If portfolio lasts for target_years, reduce withdrawal rate
                withdrawal_rate += tolerance
            else:  # If portfolio doesn't last, return previous withdrawal rate
                return withdrawal_rate - tolerance

    return {
        'ratio':ratio, 
        'origin':(prices[-1]/prices[0]), 
        'value':security_returns[-1],
        'beta': beta, 
        'alpha': alpha,
        'annual_sharp_ratio': annual_sharpe_ratio,
        'latest_recommand': strategy[-10:]
    }


print(df.tail(20))
print(buy_and_sell(df.iloc[12:]))



                     Date    Close   ratio  Moving_Average    Diff   Mark  \
1775  12/15/2023 16:00:00  4719.19  114.82       4619.3825  114.82   True   
1776  12/22/2023 16:00:00  4754.63   35.44       4668.2050   35.44   True   
1777  12/29/2023 16:00:00  4769.83   15.20       4712.0050   15.20   True   
1778    1/5/2024 16:00:00  4697.24  -72.59       4735.2225  -72.59  False   
1779   1/12/2024 16:00:00  4783.83   86.59       4751.3825   86.59   True   
1780   1/19/2024 16:00:00  4839.81   55.98       4772.6775   55.98   True   
1781   1/26/2024 16:00:00  4890.97   51.16       4802.9625   51.16   True   
1782    2/2/2024 16:00:00  4958.61   67.64       4868.3050   67.64   True   
1783    2/9/2024 16:00:00  5026.61   68.00       4929.0000   68.00   True   
1784   2/16/2024 16:00:00  5005.57  -21.04       4970.4400  -21.04   True   
1785   2/23/2024 16:00:00  5088.80   83.23       5019.8975   83.23   True   
1786    3/1/2024 16:00:00  5137.08   48.28       5064.5150   48.28   True   

/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/1730280971.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Moving_Average'] = df['Close'].rolling(window=span).mean()
/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/1730280971.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/1730280971.py:11: SettingWithCopyWarning: 
A value is trying to be 

In [36]:
{'ratio': 0.2009051019811337, 'origin': 15.002529858210272, 'value': 3.014084791138738, 'beta': 0.11909202741528965, 'alpha': 1.3269781290650928, 'annual_sharp_ratio': 9.085094808947634, 'latest_recommand': ['Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Sell', 'Hold', 'Buy']}
{'ratio': 0.33163908828838556, 'origin': 15.002529858210272, 'value': 4.975425324196137, 'beta': 0.32595653823983367, 'alpha': 0.746708608850416, 'annual_sharp_ratio': 8.676340962574303, 'latest_recommand': ['Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Buy', 'Hold', 'Sell']}



AttributeError: 'Series' object has no attribute 'value'

In [ ]:
df['Diff'] = df['Close'] - df['Close'].shift(1)

def hold_no_sell(df, annual_risk_free_rate=0.06, span=4):
    # span lookback weeks
    # risk free rate
    df['Moving_Average'] = df['Close'].rolling(window=span).mean()
    df['EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
    df['Mark'] = df['Close'] > df['EMA']
    marks = df['Mark'].values
    curr = marks[0]
    prices = df['Close'].values
    spx_returns = (df['Close'] / prices[0]).values
    base = 1.
    share = 0.
    security_returns = []
    strategy = []
    ### buy in strategy
    for mark, price in zip(marks, prices):
        if mark != curr:
            if not mark:
                share += base / price
                base = 0.
                strategy.append('Buy')
            else:
                base += share * price
                share = 0.
                strategy.append('Sell')
        else:
            strategy.append('Hold')
        curr = mark
        security_returns.append(base + share * price)
    ratio = (base + share * prices[-1]) / (prices[-1] / prices[0])

    # Calculate the mean returns
    security_mean_return = np.mean(security_returns)
    spx_mean_return = np.mean(spx_returns)

    # Calculate the covariance and variance of returns
    covariance = np.cov(security_returns, spx_returns)[0, 1]
    variance_spx = np.var(spx_returns)

    # Calculate beta
    beta = covariance / variance_spx

    # Calculate alpha
    alpha = security_mean_return - (beta * spx_mean_return)

    # Calculate the excess return of the security over the risk-free rate
    # Sample weekly returns of the security and risk-free rate
    annual_security_returns =  np.array([y/x for x, y in zip(security_returns[:-52], security_returns[52:])])

    # Calculate the excess return of the security over the risk-free rate
    excess_returns = annual_security_returns - annual_risk_free_rate

    # Calculate the annualized average excess return
    annualized_excess_return = np.mean(excess_returns)  # Approximate number of weeks in a year

    # Calculate the annualized standard deviation of returns
    annualized_std_dev = np.std(annual_security_returns) # Approximate number of weeks in a year

    # Calculate the annual Sharpe Ratio
    annual_sharpe_ratio = annualized_excess_return / annualized_std_dev

    def calculate_maximum_withdrawal_rate(weekly_returns, target_years, tolerance=1e-6):
        withdrawal_rate = 0.01  # Starting withdrawal rate
        while True:
            # Calculate portfolio balance after target_years with current withdrawal rate
            balance = calculate_portfolio_balance(weekly_returns, withdrawal_rate)
            if balance > 0:  # If portfolio lasts for target_years, reduce withdrawal rate
                withdrawal_rate += tolerance
            else:  # If portfolio doesn't last, return previous withdrawal rate
                return withdrawal_rate - tolerance

    return {
        'ratio':ratio, 
        'origin':(prices[-1]/prices[0]), 
        'value':security_returns[-1],
        'beta': beta, 
        'alpha': alpha,
        'annual_sharp_ratio': annual_sharpe_ratio,
        'latest_recommand': strategy[-10:]
    }


print(df.tail(20))
print(buy_and_sell(df.iloc[12:]))

